# Optimization of OT2 transfer paramters of viscous liquids guided by Baysesian Optimization

## Summary

This notebook objective is to generate new suggestions of aspiration and dispense rates that will minimize the tansfer error while minimizing the time of transfer of a viscous liquid. The code in this notebook will aim to optimize the liquid transfer parameters  to minimize the mean error from three volumes (default 300, 500, 1000) and the time of transfer to dispense 1000 uL.
The code is strucutred as follows:
1.  Fisrt section is for importing the relevant packages to perform BO, inclduing Pytorch and BOtorch
2.  Second section includes the definition of the BO_LiqTransfer class that includes the method optimized_suggestions() that generates BO optimized aspiration and dispense rate values for a particular data set.
3.  Third section includes the code to run in conjuction with the OT2 viscous liquid transfer notebook. The steps for the optimziation are:

    i. Initilize a BO_LiTransfer objecet and load initilization data using data_from_csv() method

    ii. Run optimized_suggestions() method

    iii. Run liquid transfer using the best suggestion for aspiration and dispense rates in OT2 notebook

    iv. Update latest %error obtained from the transfer using suggested aspiration and dispense rates.
    
    v. Iterate through steps ii-iV
 

# 1. Imports

In [2]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time
from datetime import date

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.objective import IdentityMCObjective
from botorch.acquisition import qNoisyExpectedImprovement

# utilities
from botorch.optim.optimize import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

from gpytorch.constraints import GreaterThan
from torch.optim import SGD
from gpytorch.mlls import ExactMarginalLogLikelihood

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)


# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


# 2. BO_LiqTransfer class

In [3]:
class BO_LiqTransfer:

    def __init__(self, liquid_name):
        self.liquid_name = liquid_name
        self._data = None
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error']
        self.bmax = 1.25
        self.bmin = 0.2
        self._latest_suggestion = None
        self._latest_volume = None
        self._latest_acq_value = None
        self.mean_volumes = [100,500,1000]
    
   
    def set_data(self,df):
        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']
        if 'acq_value' not in df.columns:
            df['acq_value'] = None

        if df.loc[:,self.features].duplicated().sum()==0:
            df_mean = df
        else:
            df_duplicates = df.where(df.duplicated(self.features,keep=False)==True).dropna(how='all')
            df_incomplete = df.where(df.duplicated(self.features,keep=False)==False).dropna(how='all')
            df_mean = pd.DataFrame(columns= df.columns)
            for index,values in df_duplicates.drop_duplicates(self.features).iterrows():
                if len(df_duplicates.loc[index:index+2]) == len(self.mean_volumes):
                    mean_error =df_duplicates.loc[index:index+2,'%error'].abs().mean()
                    df_duplicates.loc[index,'%error'] = -mean_error
                    df_duplicates.loc[index, 'volume'] ='mean'+str(self.mean_volumes)
                    df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index]]])
                else:
                    df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
            df_mean = pd.concat([df_mean,df_incomplete])
            df_mean = df_mean.reset_index(drop=True)    
        self._data = df_mean
 
        




    def data_from_csv(self,file_name):
        data = pd.read_csv(file_name)
        data = data.loc[:,['liquid','pipette','volume','aspiration_rate','dispense_rate','blow_out_rate','delay_aspirate','delay_dispense','delay_blow_out','%error']]
        self.set_data(data)


    def update_data(self,error,volume= 1000):
        self._latest_volume = volume
        updated_data = pd.concat([self._data,self._data.iloc[[-1]]],ignore_index=True)
        updated_data.loc[updated_data.last_valid_index(),'volume'] = self._latest_volume
        updated_data.loc[updated_data.last_valid_index(),'aspiration_rate']  = self._latest_suggestion['aspiration_rate']
        updated_data.loc[updated_data.last_valid_index(),'dispense_rate']  = self._latest_suggestion['dispense_rate']
        updated_data.loc[updated_data.last_valid_index(),'%error'] = error
        self.set_data(updated_data)
        return self._data
                                

    def xy_split(self):
        df_train = self._data.where(self._data['volume']=='mean'+str(self.mean_volumes)).dropna(how='all')
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train

    def set_bounds(self, x_train):
        return torch.vstack([x_train[0]*self.bmin, x_train[0]*self.bmax])



    def fit_surrogate(self):
        x_train, y_train = self.xy_split()
        x_train = torch.tensor(x_train.to_numpy(dtype=float), **tkwargs)
        y_train = torch.tensor(y_train.to_numpy(dtype=float), **tkwargs)
        y_train = -torch.absolute(y_train)
        

        problem_bounds = self.set_bounds(x_train)
       
        train_x_gp = normalize(x_train, problem_bounds)
        model1 = SingleTaskGP(train_x_gp, y_train, outcome_transform=Standardize(m=1))
        mll1 = ExactMarginalLogLikelihood(model1.likelihood, model1)

        fit_gpytorch_model(mll1)
    
        return model1, train_x_gp, y_train, problem_bounds
    
    def optimized_suggestions(self, random_state= 42):
        if random_state != None:
            torch.manual_seed(random_state) 
        standard_bounds = torch.zeros(2, len(self.features), **tkwargs)
        standard_bounds[1] = 1
        model1, train_x_gp, y_train, problem_bounds = self.fit_surrogate()
        acq_func1 = qNoisyExpectedImprovement(
        model=model1,
        X_baseline=train_x_gp, # feed total list of train_x for this current iteration
        sampler=SobolQMCNormalSampler(sample_shape=512),
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on  

        sobol1 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol2 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol_all = torch.vstack([sobol1, sobol2])
            
        acq_value_list = []
        for i in range(0, sobol_all.shape[0]):
            with torch.no_grad():
                acq_value = acq_func1(sobol_all[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())
                
        # filter the best 12 QMC candidates first
        sorted_x = sobol_all.cpu().numpy()[np.argsort((acq_value_list))]
        qnei_x = torch.tensor(sorted_x[-12:], **tkwargs)  
        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(qnei_x, bounds=problem_bounds)
        new_x = pd.DataFrame(new_x.numpy(),columns=['aspiration_rate','dispense_rate'])
        new_x['acq_values'] = sorted(acq_value_list, reverse=True)[:12]
        self._latest_suggestion = new_x[['aspiration_rate','dispense_rate']].iloc[0]
        return new_x
        





        
    
    


# 3. Guided BO optimization
i.   Create BO_LiqTransfer object and load data set.

Please set liquid name and volume to transfer according to the experiment.

In [28]:
# Change according to experiment
liquid_name = 'Viscosity_std_505' 

# Do not change
liq = BO_LiqTransfer(liquid_name)
liq.data_from_csv(r'Opentrons_experiments\BOTorch_optimization\CCF_initialization\BOTorch_optimization_CCF_Viscosity_std_1275.csv')
liq._data

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000,acq_value
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.657700,None
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.657700,None
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.657700,None
3,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",22.061005,22.061005,0,5,5,0,-27.408934,100.657700,None
4,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.526160,None
5,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.526160,None
6,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.526160,None
7,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",27.576257,27.576257,0,5,5,0,-30.110399,82.526160,None
8,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578,None
9,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578,None



ii.   Run BO_LiqTransfer.optimized_suggestions() method to obtain optimized aspiration and dispense rates.



In [45]:
liq.optimized_suggestions()

c:\ProgramData\Anaconda3\lib\site-packages\botorch\models\utils\assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)


,aspiration_rate,dispense_rate,acq_values
0,7.426019,4.671498,0.449173
1,6.571726,7.240593,0.392742
2,5.057177,9.053698,0.388766
3,6.056770,8.495537,0.376736
4,6.480782,7.234483,0.361719
5,6.191980,7.747433,0.336226
6,5.250680,6.873120,0.321214
7,5.606402,7.368671,0.283577
8,4.416201,6.749344,0.280289
9,4.863199,8.372338,0.277004


iii. Run liquid transfer using the best suggestion for aspiration and dispense rates in OT2 notebook.

iv. Update volume and latest %error obtained from the transfer using suggested aspiration and dispense rates. Repeat for 1000, 500, 100 uL

In [48]:
volume= 300
liq.update_data(-3.807998,volume)



,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000,acq_value
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.6577,None
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.6577,None
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.6577,None
3,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",22.061005,22.061005,0,5,5,0,-27.408934,100.6577,None
4,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.52616,None
5,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.52616,None
6,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.52616,None
7,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",27.576257,27.576257,0,5,5,0,-30.110399,82.52616,None
8,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578,None
9,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578,None


In [50]:
#save after each standard-experiment iteration
liq._data.to_csv('C:\\Users\\amdm_\\OneDrive\\Documents\\GitHub\\viscosity_liquid_transfer_Pablo\\Opentrons_experiments\\BOTorch_optimization\\VS_code_csv\\'+liquid_name+'_'+'exp3_completeda1275.csv', index = False)

v. Iterate trhough last two code cells.

In [28]:
liq._data

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000,acq_value
0,Viscosity_std_505,p1000,1000,49.507284,49.507284,0,5,5,0,-7.382241,50.398096,None
1,Viscosity_std_505,p1000,500,49.507284,49.507284,0,5,5,0,-8.280548,50.398096,None
2,Viscosity_std_505,p1000,300,49.507284,49.507284,0,5,5,0,-8.556950,50.398096,None
3,Viscosity_std_505,p1000,"mean[100, 500, 1000]",49.507284,49.507284,0,5,5,0,-8.073247,50.398096,None
4,Viscosity_std_505,p1000,1000,61.884105,4.950728,0,5,5,0,-0.345503,228.149719,None
5,Viscosity_std_505,p1000,500,61.884105,4.950728,0,5,5,0,-2.360935,228.149719,None
6,Viscosity_std_505,p1000,300,61.884105,4.950728,0,5,5,0,-2.760183,228.149719,None
7,Viscosity_std_505,p1000,"mean[100, 500, 1000]",61.884105,4.950728,0,5,5,0,-1.822207,228.149719,None
8,Viscosity_std_505,p1000,1000,4.950728,4.950728,0,5,5,0,2.810089,413.980961,None
9,Viscosity_std_505,p1000,500,4.950728,4.950728,0,5,5,0,8.372682,413.980961,None
